#Stability of Householder Triangularization
This is a reproduction of the example from the beginning of Lecture 16 of *Numerical Linear Algebra* by Trefethen & Bau.

In [ ]:
import numpy as np

def householder(A):
    """QR factorization via Householder triangularization."""
    m, n = A.shape
    V = np.zeros(A.shape)
    R = A.copy()
    for k in range(n-1):
        x = R[k:,k].copy()
        x[0] = x[0] + np.sign(x[0])*np.linalg.norm(x,2)
        x = x/np.linalg.norm(x,2)
        V[k:,k] = x.copy()
        for j in range(k,n):
            R[k:,j] = R[k:,j] - 2*V[k:,k]*np.dot(V[k:,k].T,R[k:,j])
    return V,R[:n,:]

def apply_Q(V,x):
    """Algorithm 10.3 of Trefethen & Bau."""
    m, n = V.shape
    for k in range(n-1,-1,-1):
        x[k:] = x[k:] - 2*np.dot(V[k:,k],x[k:])*V[k:,k]
    return x

def compute_Q(V):
    m, n = V.shape
    Q = np.zeros((m,n))
    for k in range(n):
        x = np.zeros(m)
        x[k] = 1.
        Q[:,k] = apply_Q(V,x)
    return Q

We're going to investigate the accuracy of the QR factorization generated by Householder triangularization.  In order to do so, we construct a matrix $A$ with a known QR factorization by creating a random upper-triangular matrix and a random unitary matrix:

In [ ]:
N = 40
R = np.triu(np.random.randn(N,N))
Q, X = np.linalg.qr(np.random.randn(N,N))
A = np.dot(Q,R)

Now we compute the QR factorization of $A$:

In [ ]:
#Q2, R2 = np.linalg.qr(A)
V, R2 = householder(A)
Q2 = compute_Q(V)

...and check how close the factors are to the exact ones:

In [ ]:
print np.linalg.norm(Q-Q2)
print np.linalg.norm(R-R2)

The accuracy is extremely poor!  But it's all we can expect based on the condition number of $A$:

In [ ]:
print np.linalg.cond(A)

The errors above are *forward errors*.  But what if we multiply our factors back together and compare them to $A$?  That difference is the *residual*, or *backward error*:

In [ ]:
A2 = np.dot(Q2,R2)
np.linalg.norm(A-A2)

It's extremely accurate!  Somehow all the errors cancel out, to very high precision.

This cancellation is extremely lucky, and would never happen if the errors were random:

In [ ]:
Q3 = Q + 1e-4*np.random.randn(N,N)
R3 = R + 1e-4*np.random.randn(N,N)
A3 = np.dot(Q3,R3)
print np.linalg.norm(A-A3)

In [ ]:
for i in range(Q2.shape[1]):
    if Q2[0,i]*Q[0,i] < 0:
        Q2[:,i] = -Q2[:,i]
        R2[i,:] = -R2[i,:]
print np.linalg.norm(Q-Q2)
print np.linalg.norm(R-R2)